In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [ ]:
# https://archive.ics.uci.edu/dataset/59/letter+recognition
# first used in "Letter Recognition Using Holland-Style Adaptive Classifiers"
from ucimlrepo import fetch_ucirepo 

# fetch dataset 
letter_recognition = fetch_ucirepo(id=59) 
  
# data (as pandas dataframes) 
X_data = letter_recognition.data.features 
y_data = letter_recognition.data.targets

In [ ]:
X = X_data.to_numpy()
y = y_data.to_numpy().flatten()

In [ ]:
def map_letter_array_to_numbers(letters):
    letter_to_number = {chr(i): i - ord('A') + 1 for i in range(ord('A'), ord('Z') + 1)}
    return np.array([letter_to_number.get(letter, 0) for letter in letters])

def map_number_array_to_letters(numbers):
    number_to_letter = {int(c): chr(c + ord('A') - 1) for c in range(26)}
    return np.array([number_to_letter.get(number, 'A') for number in numbers])    

In [ ]:
y_num = map_letter_array_to_numbers(y)

In [ ]:
# Get 60% of the dataset as the training set. Put the remaining 40% in temporary variables: x_ and y_.
x_train, x_, y_train, y_ = train_test_split(X, y_num, test_size=0.40, random_state=1)

# Split the 40% subset above into two: one half for cross validation and the other for the test set
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

# Delete temporary variables
del x_, y_

In [ ]:
print(x_train.shape)
print(x_train[0]) # note the values are all on the same order, so I skip normalization
print(y[0])
print(y_train[0])

In [ ]:
model_1 = Sequential(
    [
        tf.keras.Input(shape=(16,)),
        Dense(16, activation = 'relu'),
        Dense(27, activation = 'linear')
    ],
    name='model_1'
)
model_2 = Sequential(
    [
        tf.keras.Input(shape=(16,)),
        Dense(100, activation = 'relu'),
        Dense(60, activation = 'relu'),
        Dense(27, activation = 'linear')
    ],
    name='model_2'
)
model_3 = Sequential(
    [
        tf.keras.Input(shape=(16,)),
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(27, activation = 'linear')
    ],
    name='model_3'
)
nn_models = [model_1, model_2, model_3]

In [ ]:
def classification_error(model, x, y):
    yhat_all_features = model.predict(x)
    yhat = np.zeros(yhat_all_features.shape[0])
    for i in range(yhat_all_features.shape[0]):
        yhat[i] = np.argmax(yhat_all_features[i])
    return np.mean(yhat != y)

In [ ]:
# Initialize lists that will contain the errors for each model
nn_train_error = []
nn_cv_error = []

# Loop over the the models
for model in nn_models:
    # Setup the loss and optimizer
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    )
    print(f"Training {model.name}...")
    
    # Train the model
    model.fit(
        x_train, 
        y_train,
        epochs=50,
        verbose=0
    )
    print("Done!\n")
    
    # Record the fraction of misclassified examples for the training set
    train_error = classification_error(model, x_train, y_train)
    nn_train_error.append(train_error)
    
    # Record the fraction of misclassified examples for the cross validation set
    cv_error = classification_error(model, x_cv, y_cv)
    nn_cv_error.append(cv_error)

In [ ]:
# Print the error results
for model_num in range(len(nn_train_error)):
    print(
        f"Model {model_num+1} ({nn_models[model_num].name}): Training Set Classification Error: {nn_train_error[model_num]:.5f}, " +
        f"CV Set Classification Error: {nn_cv_error[model_num]:.5f}"
        )

In [ ]:
# Select the model with the lowest error
model_num = np.argmin(nn_cv_error)
model = nn_models[model_num]
print(f"Model {model_num+1} ({model.name}) has the lowest error")

In [ ]:
# Compute the test error
nn_test_error = classification_error(model, x_train, y_train)
print(f"Test error: {nn_test_error}")

In [ ]:
model.save('letter_recognition_model.h5')

In [ ]:
prediction = model.predict(x_test[:1])
yhat = np.argmax(prediction)
print(yhat)
print(y_test[0])